# xlm roberta language detection

This is the notebook that was used to train the model xlm-roberta-base-language-detection, which is a fine-tuned version of [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) on the [Language Identification](https://huggingface.co/datasets/papluca/language-identification) dataset.

In [ ]:
!pip install -q datasets transformers[sentencepiece] langid watermark

Let's check the GPU we've got (if we have one):

In [ ]:
!nvidia-smi

Sat Nov  5 17:06:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Exact versions of the libraries:

In [ ]:
%load_ext watermark
%watermark -p torch,datasets,sklearn,transformers,langid

torch       : 1.12.1+cu113
datasets    : 2.6.1
sklearn     : 1.0.2
transformers: 4.24.0
langid      : 1.1.6



In [ ]:
import time
from pathlib import Path

import langid
import torch
from datasets import load_dataset
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding, 
    pipeline,
    Trainer,
    TrainingArguments
)

This is the folder in which we'll store model checkpoints (we assume it's a place in our Google Drive):

In [ ]:
gdrive_dir = Path('/content/drive/MyDrive/final_year_project_folder/')

### Get and split the dataset

In [ ]:
dataset = load_dataset("papluca/language-identification")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/papluca___csv/papluca--language-identification-b9299393bab34ec8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ds_train = dataset['train']
ds_valid = dataset['validation']
ds_test = dataset['test']

print(f"Train / valid / test samples: {len(ds_train)} / {len(ds_valid)} / {len(ds_test)}")

Train / valid / test samples: 70000 / 10000 / 10000


## Tokenization

We'll set up a tokenizer that truncates sequences longer than 128 tokens, but ignore padding for now. This is because we'll use dynamic padding, i.e. we'll pad to the length of the *longest sequence in each batch*.

In [ ]:
model_ckpt = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
def tokenize_text(sequence):
    """Tokenize input sequence."""
    return tokenizer(sequence["text"], truncation=True, max_length=128)

Tokenize all sub-datasets:

In [ ]:
tok_train = ds_train.map(tokenize_text, batched=True)
tok_valid = ds_valid.map(tokenize_text, batched=True)
tok_test = ds_test.map(tokenize_text, batched=True)

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

Prepare forward and backward mappings between labels strings and integers:

In [ ]:
amazon_languages = ['en', 'de', 'fr', 'es', 'ja', 'zh']
xnli_languages = ['ar', 'el', 'hi', 'ru', 'th', 'tr', 'vi', 'bg', 'sw', 'ur']
stsb_languages = ['it', 'nl', 'pl', 'pt']

all_langs = sorted(list(set(amazon_languages + xnli_languages + stsb_languages)))

In [ ]:
id2label = {idx: all_langs[idx] for idx in range(len(all_langs))}
label2id = {v: k for k, v in id2label.items()}
label2id

{'ar': 0,
 'bg': 1,
 'de': 2,
 'el': 3,
 'en': 4,
 'es': 5,
 'fr': 6,
 'hi': 7,
 'it': 8,
 'ja': 9,
 'nl': 10,
 'pl': 11,
 'pt': 12,
 'ru': 13,
 'sw': 14,
 'th': 15,
 'tr': 16,
 'ur': 17,
 'vi': 18,
 'zh': 19}

In [ ]:
def encode_labels(example):
    """Map string labels to integers."""
    example["labels"] = label2id[example["labels"]]
    return example

Encode targets:

In [ ]:
tok_train = tok_train.map(encode_labels, batched=False)
tok_valid = tok_valid.map(encode_labels, batched=False)
tok_test = tok_test.map(encode_labels, batched=False)

  0%|          | 0/70000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/10000 [00:00<?, ?ex/s]

Let's have a look at the corpus statistics to see e.g. whether our truncation length is fine.

In [ ]:
from statistics import mean, stdev

_len = [len(sample) for sample in tok_train['input_ids']]
avg_len, std_len = mean(_len), stdev(_len)
min_len, max_len = min(_len), max(_len)

print('-'*10 + ' Corpus statistics ' + '-'*10)
print(f'\nAvg. length: {avg_len:.1f} (std. {std_len:.1f})')
print('Min. length:', min_len)
print('Max. length:', max_len)

---------- Corpus statistics ----------

Avg. length: 33.9 (std. 24.6)
Min. length: 3
Max. length: 128


Max. length is 128, but on average samples are much shorter (ca. 34 tokens). Padding batches

In [ ]:
# Use dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Model training

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
  model_ckpt, num_labels=len(all_langs), id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

We define here the metrics that we're going to monitor during training:

In [ ]:
def compute_metrics(pred):
    """Custom metric to be used during training."""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)  # Accuracy
    f1 = f1_score(labels, preds, average="weighted")  # F1-score
    return {
        "accuracy": acc,
        "f1": f1
        }

To train our model, we'll use the HF `Trainer`. The 1st step is to create an instance of the `TrainingArguments` class, which will contain all the hyperparameters the `Trainer` will use for training and evaluation.

In [ ]:
epochs = 2
lr = 2e-5
train_bs = 64
eval_bs = train_bs * 2

# Log training loss at each epoch
logging_steps = len(tok_train) // train_bs
# Out dir
output_dir = gdrive_dir / "xlm-roberta-base-finetuned-language-detection"

training_args = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=epochs,
  learning_rate=lr,
  per_device_train_batch_size=train_bs,
  per_device_eval_batch_size=eval_bs,
  evaluation_strategy="epoch",
  logging_steps=logging_steps,
  fp16=True,  # Remove if GPU doesn't support it
)

Then, we can instantiate the `Trainer`:

In [ ]:
trainer = Trainer(
  model,
  training_args,
  compute_metrics=compute_metrics,
  train_dataset=tok_train,
  eval_dataset=tok_valid,
  data_collator=data_collator,
  tokenizer=tokenizer,
)

Using cuda_amp half precision backend


Let's train the model! 🚀

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 70000
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2188
  Number of trainable parameters = 278059028
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.248100,0.014772,0.996800,0.996815
2,0.009400,0.008678,0.998000,0.998000


Saving model checkpoint to drive/MyDrive/Colab Notebooks/HuggingFace_course/HF_course_community_event/xlm-roberta-base-finetuned-language-detection/checkpoint-500
Configuration saved in drive/MyDrive/Colab Notebooks/HuggingFace_course/HF_course_community_event/xlm-roberta-base-finetuned-language-detection/checkpoint-500/config.json
Model weights saved in drive/MyDrive/Colab Notebooks/HuggingFace_course/HF_course_community_event/xlm-roberta-base-finetuned-language-detection/checkpoint-500/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/Colab Notebooks/HuggingFace_course/HF_course_community_event/xlm-roberta-base-finetuned-language-detection/checkpoint-500/tokenizer_config.json
Special tokens file saved in drive/MyDrive/Colab Notebooks/HuggingFace_course/HF_course_community_event/xlm-roberta-base-finetuned-language-detection/checkpoint-500/special_tokens_map.json
Saving model checkpoint to drive/MyDrive/Colab Notebooks/HuggingFace_course/HF_course_community_event/xlm-rober

TrainOutput(global_step=2188, training_loss=0.12860834635892063, metrics={'train_runtime': 1176.8649, 'train_samples_per_second': 118.96, 'train_steps_per_second': 1.859, 'total_flos': 8561635871314944.0, 'train_loss': 0.12860834635892063, 'epoch': 2.0})

### Our model

In [ ]:
device = 0 if torch.cuda.is_available() else -1

model_ckpt = "victoria-nyamai/xlm-roberta-base-language-detection"
pipe = pipeline("text-classification", model=model_ckpt, device=device)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
start_time = time.perf_counter()
model_preds = [s['label'] for s in pipe(ds_test.text.values.tolist(), truncation=True, max_length=128)]
print(f"{time.perf_counter() - start_time:.2f} seconds")

106.06 seconds


Classification report for **our model**:

In [ ]:
print(classification_report(ds_test.labels.values.tolist(), model_preds, digits=3))

              precision    recall  f1-score   support

          ar      1.000     0.998     0.999       500
          bg      0.992     1.000     0.996       500
          de      1.000     1.000     1.000       500
          el      1.000     1.000     1.000       500
          en      0.988     1.000     0.994       500
          es      0.996     0.996     0.996       500
          fr      1.000     1.000     1.000       500
          hi      0.967     1.000     0.983       500
          it      0.994     0.992     0.993       500
          ja      1.000     1.000     1.000       500
          nl      1.000     0.994     0.997       500
          pl      1.000     0.992     0.996       500
          pt      0.998     0.996     0.997       500
          ru      0.998     0.996     0.997       500
          sw      0.994     0.992     0.993       500
          th      1.000     1.000     1.000       500
          tr      0.996     1.000     0.998       500
          ur      0.998    